In [22]:
# 79-character line limit
######### ######### ######### ######### ######### ######### ######### #########
%reset -f

import numpy as np # The NumPy library
#import math # The math module, np includes it
from scipy.integrate import quad # Method for integration in scipy.integrate sub-package


cells_num = 32

# The sizes are for UQs for number of cells: 4, 8, 16, 32, 64, 128, 256.

# The opitimized spaces for the Guassian distribution in a uniform quantizer(UQ).
all_size_gaus = np.array([0.9957, 0.5860, 0.3352, 0.1881, 0.1041, 0.0569, 0.0308])

# The opitimized spaces for the Laplacian distribution in a uniform quantizer(UQ).
all_size_laplacian = np.array(
                   [1.414, 1.0873, 0.8707, 0.7309, 0.6334, 0.5613, 0.5055])


'''The opitimized spaces for a Mixed Gaussian distribution in 
a uniform quantizer(UQ). This distribution is not symmetric and the values are
for for cells number 2, 4, 8, 16, 32, 64, 128, 256'''
all_boudary1s = [-0.05, -2.02, -3.17, -3.93, -4.49, -5.08, -5.16, -5.50]
all_deltas = [4, 1.81, 1, 0.54, 0.29, 0.16, 0.08, 0.05]

'''To change the symmetric pdf, two parts of code must be changed include the 
cell_size and pdf'''

cell_size = all_size_gaus[int(np.log2(cells_num))-2]
# cell_size = all_size_laplacian[int(np.log2(cells_num))-2]

# np.arange has rounding error issue.
# boundaries_symm = np.arange(-(cells_num/2-1) * cell_size, # For symmetric distributions
#                              (cells_num/2) * cell_size,
#                             cell_size)

# Boundaries for the symmetric distributions are computed.
boundaries_symm = np.linspace(-(cells_num/2-1) * cell_size, # For symmetric distributions
                               (cells_num/2-1) * cell_size,
                              cells_num-1)

# Boundaries for the asymmetric distributions are computed.
boundary1 = all_boudary1s[int(np.log2(cells_num))-1]
delta = all_deltas[int(np.log2(cells_num))-1]

boundaries_asymm = []
for i in range (0,cells_num-1):
        boundary = boundary1 + (i) * delta
        boundaries_asymm = np.append(boundaries_asymm, boundary)
        
'''To change the symmetric pdf to asymetric pdf, two parts of code 
must be changed include the boundaries and pdf'''        

# boundaries = boundaries_symm # For symmetric distributions
boundaries = boundaries_asymm # For asymetric distributions
        

n_inf = float("-inf")
p_inf = float("inf")

boundaries = np.insert(boundaries, 0, n_inf)
boundaries = np.append(boundaries, p_inf)

def pdf(x): # Defining the distribution
#     gaus_std = 1
#     gaus_mean = 0
#     pdf = 1/(gaus_std*np.sqrt(2*np.pi)) * \
#                   np.exp(-0.5*((x-gaus_mean)/gaus_std)**2) # Gaussian pdf
    
#     div = 1
#     pdf = 1/(2*div)*np.exp(-abs(x)/div) # Laplacian pdf

    Mean1 = -2
    Mean2 = 2
    stdev1 = 1
    stdev2 = 1
    alpha = 0.7
    pdf = (alpha)*(1/np.sqrt(2*np.pi*stdev1)*np.exp(-(x-Mean1)**2/(2*stdev1**2)))\
        + (1-alpha)*(1/np.sqrt(2*np.pi*stdev2)*np.exp(-(x-Mean2)**2/(2*stdev2**2)))
    
    return pdf

def xpdf(x):
    xpdf = x * pdf(x)
    return xpdf

def x2pdf (x):
    x2pdf = x * xpdf(x)
    return x2pdf


prbs = []
xprbs = []
x2prbs = []
cell_reps = []
for i in range (0, cells_num):
    cell_prb, integ_err = quad(pdf, boundaries[i], boundaries[i+1])
    prbs = np.append(prbs, cell_prb)
    
    cell_xprb, integ_err = quad(xpdf, boundaries[i], boundaries[i+1])
    xprbs = np.append(xprbs, cell_xprb)
    
    cell_x2prb, integ_err = quad(x2pdf, boundaries[i], boundaries[i+1])
    x2prbs = np.append(x2prbs, cell_x2prb)
    
    cell_rep = cell_xprb / cell_prb
    cell_reps = np.append(cell_reps, cell_rep)
    
    
# Function definitions
def relu(x):
    return np.maximum(0,x)

def lrelu(x):
    return np.where(x > 0, x, x * 0.1)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [27]:
# Weights and biases initialization

l1_size = 32
particles_num = 1024

# We initialize the particles.
W1 = np.random.randn(particles_num, l1_size, cells_num)
B1 = np.random.randn(particles_num, l1_size)
W2 = np.random.randn(particles_num, cells_num, l1_size)

In [28]:
# First feed forward
distortions = []
for i in range (particles_num):
    l1_outs = relu(np.dot(W1[i], cell_reps)+B1[i])
    l2_outs = np.dot(W2[i], l1_outs)
    l2_outs = np.round(l2_outs, 16)

    # We assigned the indexes based on the final layer outputs.
    sorted_outs = np.sort(l2_outs)
    indexes = []
    for i in range(0, cells_num):
        index = np.where(sorted_outs == l2_outs[i])
        indexes = np.append(indexes, index)

#     print(indexes)

    yijkls = [] # Yijkls based on the assigned indexes
    dijkls = []
    for i in range(0, int(cells_num/4)):
        j = i + cells_num/4
        k = i + 2 * cells_num/4
        l = i + 3 * cells_num/4

        celli = np.where(indexes == i)
        cellj = np.where(indexes == j)
        cellk = np.where(indexes == k)
        celll = np.where(indexes == l)

        yijkl = (xprbs[celli] + xprbs[cellj] + xprbs[cellk] + xprbs[celll])\
               /(prbs[celli] + prbs[cellj] + prbs[cellk] + prbs[celll])
        yijkls = np.append(yijkls, yijkl)

        dijkl = x2prbs[celli] + yijkl**2 * prbs[celli] - yijkl *2 * xprbs[celli]\
              + x2prbs[cellj] + yijkl**2 * prbs[cellj] - yijkl *2 * xprbs[cellj]\
              + x2prbs[cellk] + yijkl**2 * prbs[cellk] - yijkl *2 * xprbs[cellk]\
              + x2prbs[celll] + yijkl**2 * prbs[celll] - yijkl *2 * xprbs[celll]
        dijkls = np.append(dijkls, dijkl)

    distortion = np.round(sum(dijkls), 16) # The Eve's distortion based on the assigned indexes

    distortions = np.append(distortions, distortion)
    
# print(distortions)
print(np.max(distortions))

gb_pos = np.argmin(-distortions) # Global best particle
gb_value = -distortions[gb_pos]

gbW1 = W1[gb_pos]
gbB1 = B1[gb_pos]
gbW2 = W2[gb_pos]

pbW1 = W1
pbB1 = B1
pbW2 = W2

pb_value = -distortions

last_VW1 = np.zeros([particles_num, l1_size, cells_num])
last_VB1 = np.zeros([particles_num, l1_size])
last_VW2 = np.zeros([particles_num, cells_num, l1_size])

tW1 = W1.copy()
tB1 = B1.copy()
tW2 = W2.copy()

4.328991277853007


In [ ]:
######### ######### ######### ######### ######### ######### ######### #########
iterations = 2000
# inertia_weight = 0.9
# c1 = 2 # cognitive_weight
# c2 = 2 # global_weight

# r1 = np.random.random()
# r2 = np.random.random()

VW1 = np.zeros([particles_num, l1_size, cells_num])
VB1 = np.zeros([particles_num, l1_size])
VW2 = np.zeros([particles_num, cells_num, l1_size])

gb_values =[]

rond = 1.5
sigma = 4
delta1 = 2.5
delta2 = 0.5

phi = 0.666
tau = 0.6

for m in range(0, iterations):
    c1 = -rond * np.arctan(m/iterations*sigma)+delta1
    c2 =  rond * np.arctan(m/iterations*sigma)+delta2
    
    inertia_weight = phi * np.cos((m/iterations)*np.pi)+tau
#     print(c1)
#     print(c2)
#     print(inertia_weight)
    print(m)
    
    for n in range(0, particles_num):
        r1 = np.random.random()
        r2 = np.random.random()
               

        # We compute the new position.
        VW1[n] = inertia_weight * last_VW1[n] + (c1 * r1 * (pbW1[n]-tW1[n]))\
             + (c2 * r2 * (gbW1-tW1[n]))
        tW1[n] = tW1[n] + VW1[n]
        last_VW1[n] = VW1[n] 
                 
        VB1[n] = (inertia_weight * last_VB1[n]) + (c1 * r1 * (pbB1[n]-tB1[n]))\
              + (c2 * r2 * (gbB1 - tB1[n]))
        tB1[n] = tB1[n] + VB1[n]
        last_VB1[n] = VB1[n]
                 
        VW2[n] = (inertia_weight * last_VW2[n]) + (c1 * r1 * (pbW2[n]-tW2[n]))\
              + (c2 * r2 * (gbW2 - tW2[n]))
        tW2[n] =tW2[n] + VW2[n]
        last_VW2[n] = VW2[n] 
        
        l1_outs = []
        l2_outs = []
        # Now we compute the new distortion.
        l1_outs = relu(np.dot(tW1[n], cell_reps)+tB1[n])
        l2_outs = np.dot(tW2[n], l1_outs)
        l2_outs = np.round(l2_outs, 16)
        
        for p in range(0, cells_num-1):
            for q in range (p+1, cells_num):
                if l2_outs[p] == l2_outs[q]:
                    l2_outs[q] += np.random.rand(1,1)-0.5

        sorted_outs = np.sort(l2_outs)
        indexes = []
        for i in range(0, cells_num):
            index = np.where(sorted_outs == l2_outs[i])
            indexes = np.append(indexes, index)
            
        yijkls = [] # Yijkls based on the assigned indexes
        dijkls = []
        for i in range(0, int(cells_num/4)):
            j = i + cells_num/4
            k = i + 2 * cells_num/4
            l = i + 3 * cells_num/4

            celli = np.where(indexes == i)
            cellj = np.where(indexes == j)
            cellk = np.where(indexes == k)
            celll = np.where(indexes == l)

            yijkl = (xprbs[celli] + xprbs[cellj] + xprbs[cellk] + xprbs[celll])\
                   /(prbs[celli] + prbs[cellj] + prbs[cellk] + prbs[celll])
            yijkls = np.append(yijkls, yijkl)

            dijkl = x2prbs[celli] + yijkl**2 * prbs[celli] - yijkl *2 * xprbs[celli]\
                  + x2prbs[cellj] + yijkl**2 * prbs[cellj] - yijkl *2 * xprbs[cellj]\
                  + x2prbs[cellk] + yijkl**2 * prbs[cellk] - yijkl *2 * xprbs[cellk]\
                  + x2prbs[celll] + yijkl**2 * prbs[celll] - yijkl *2 * xprbs[celll]
            dijkls = np.append(dijkls, dijkl)

        distortion = np.round(sum(dijkls), 16) # The Eve's distortion based on the assigned indexes
            
                        
        if -distortion < pb_value[n]:
            pbW1[n] = W1[n]
            pbB1[n] = B1[n]
            pbW2[n] = W2[n]
            pb_value[n] = -distortion
        
        if -distortion < gb_value:
            gbW1 = W1[n]
            gbB1 = B1[n]
            gbW2 = W2[n]
            gb_value = -distortion
            gb_values = np.append(gb_values, gb_value)
            best_indexes = indexes
            
            print(-gb_value)        
        #print(distortion)
#         if distortion >= 1:
#             print(indexes)

print(best_indexes)

quads = []
for i in range(int(cells_num/4)):
    quad_cell1 = np.where(best_indexes==i)
    quad_cell2 = np.where(best_indexes==i+cells_num/4)
    quad_cell3 = np.where(best_indexes==i+2*cells_num/4)
    quad_cell4 = np.where(best_indexes==i+3*cells_num/4)
    quad = [quad_cell1, quad_cell2, quad_cell3, quad_cell4]
    quads = np.append(quads, quad)

quads = np.transpose(np.reshape(quads,[int(cells_num/4), 4]))
print('\n', quads)

4.330640374204505
4.339194671880209
4.341626489234101
